In [ ]:
import sys

sys.path.append('..')

import torch
import torch.nn as nn
from torchvision import transforms
from timm import create_model

import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
from pathlib import Path
import numpy as np

from src.trainer import Trainer
from src.dataset import HumanPosesDataset
from sklearn.model_selection import train_test_split

In [ ]:
import plotly.io as pio
pio.renderers.default = "browser"

# Датасет

In [ ]:
mean = [0.4638, 0.4522, 0.4148]
std = [0.2222, 0.2198, 0.2176]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.TrivialAugmentWide(),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.25),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
CSV_PATH = Path("../data/human_poses_data/train_answers.csv")
TRAIN_DIR = Path("../data/human_poses_data/img_train")

df = pd.read_csv(CSV_PATH)

train_ids, val_ids = train_test_split(
    df['img_id'].values,
    test_size=0.2,
    stratify=df['target_feature'],
    random_state=42
)

train_df = df[df['img_id'].isin(train_ids)].reset_index(drop=True)
val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)

train_dataset = HumanPosesDataset(
    data_df=train_df,
    img_dir=TRAIN_DIR,
    transform=train_transform,
)

val_dataset = HumanPosesDataset(
    data_df=val_df,
    img_dir=TRAIN_DIR,
    transform=val_transform,
)



train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

In [ ]:
num_classes = len(np.unique(df['target_feature']))
print(f"Количество классов: {num_classes}")

# Модель

In [ ]:
from src.models.teacher import ConvNeXtTeacher

model = ConvNeXtTeacher(num_classes=num_classes)

In [ ]:
from torch.amp import autocast, GradScaler

model = ConvNeXtTeacher(num_classes=16).cuda()
model.eval()

dummy = torch.randn(4, 3, 224, 224).cuda()
scaler = GradScaler()

with autocast(device_type='cuda'):
    with torch.no_grad():
        out = model(dummy)
print("Output shape:", out.shape)  # [4, 16]

# Голова

In [ ]:
NUM_EPOCH = 10

model = ConvNeXtTeacher(num_classes=16).cuda()
model.backbone.requires_grad_(False)
model.train()

optimizer = torch.optim.AdamW(
    model.head.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=NUM_EPOCH
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = GradScaler()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

In [ ]:
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=None,
    experiment_name="teacher",
    use_wandb=True,
    seed=42,
    scaler=scaler,
)

history = trainer.train()

# Фулл

In [ ]:
from src.utils import load_best_model
from src.models.teacher import ConvNeXtTeacher
from torch.amp import autocast, GradScaler

NUM_EPOCH = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = ConvNeXtTeacher(num_classes=16).cuda()
model = load_best_model(model, 'checkpoints/teacher_best.pth', device=device)

model.backbone.requires_grad_(True)
model.train()

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4
)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-4,
    steps_per_epoch=len(train_loader),
    epochs=NUM_EPOCH,
    pct_start=0.1,
    anneal_strategy='cos',
    div_factor=10.0,
    final_div_factor=1e3
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.3)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="teacher_full",
    use_wandb=True,
    seed=42,
    scaler=scaler,
)

history = trainer.train()

там какой то  потолок, я перезапущу модель и попробую понять какие классы она путает

In [ ]:
from src.utils import evaluate_model, load_best_model
from src.models.teacher import ConvNeXtTeacher

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = ConvNeXtTeacher(num_classes=16).cuda()
load_best_model(model, 'checkpoints/teacher_full_best.pth', device=device)

evaluate_model(model, val_loader, device=device)

финальное дообучение до 0.9 ф1

In [ ]:
from src.utils import load_best_model
from src.models.teacher import ConvNeXtTeacher
from torch.amp import autocast, GradScaler

NUM_EPOCH = 25

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = ConvNeXtTeacher(num_classes=16).cuda()
model = load_best_model(model, 'checkpoints/teacher_full_best.pth', device=device)

model.backbone.requires_grad_(True)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCH)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scaler = GradScaler()

In [ ]:
from src.utils import MixupCutMixAugmenter

mixup_cutmix_fn = MixupCutMixAugmenter(alpha=1.0, p_mixup=0.3)

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=NUM_EPOCH,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    batch_augment_fn=mixup_cutmix_fn,
    experiment_name="teacher_full_finalmaybe",
    use_wandb=True,
    seed=42,
    scaler=scaler,
)

history = trainer.train()

Сабмит

In [ ]:
from pathlib import Path
import pandas as pd

TEST_DIR = Path("../data/human_poses_data/img_test")

test_ids = [f.stem for f in TEST_DIR.glob("*.jpg")]
test_df = pd.DataFrame({"img_id": test_ids})


test_dataset = HumanPosesDataset(
    data_df=test_df,
    img_dir=TEST_DIR,
    transform=val_transform,
    preload=False,
    mode='test',
)


test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

In [ ]:
from src.models.teacher import ConvNeXtTeacher
from src.utils import load_best_model, make_submission

index_to_class = train_dataset.index_to_class

model = ConvNeXtTeacher(num_classes=num_classes)

load_best_model(model, 'checkpoints/teacher_full_finalmaybe_best.pth', device=device)

make_submission(model, test_loader, device=device, index_to_class=index_to_class)

(это чтобы скрыть свои скоры от других(ну там другие команды также сделали), отправлять эту модель в приватный лидерборд я не буду)

In [ ]:
!kaggle competitions submit -c ml-intensive-yandex-academy-spring-2025 -f submission.csv -m "Message"